### Semantic Skill Selection
Another approach, Semantic Skill Selection, uses semantic representations to identify and select the relevant skill based on their semantic similarity to the task requirements. Ahead of time, each skill definition and description is embedded using an encoder-only model, such as OpenAI’s ada model, Amazon’s Titan model, Cohere’s Embed model, BERT, or others. These skills are then indexed in a lightweight vector database. At runtime, the current context is then embedded using the same embedding model, a search is performed on the database, and the top skill is selected. The skill is then parameterized with a text completion model, invoked, and the response is used to compose the response for the user. This is the most common pattern, and is recommended for most use cases. It’s typically faster than Generative Skill Selection, performant, and reasonably scalable.

In [30]:
import os
import requests
import logging
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.vectorstores import FAISS
import faiss
import numpy as np
from dotenv import load_dotenv
import os
import json

In [31]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv("OPENAI_API_KEY")

In [32]:
# Initialize OpenAI embeddings and LLM
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
llm = ChatOpenAI(api_key=api_key)


In [33]:
# Tool descriptions
tool_descriptions = {
    "query_wolfram_alpha": "Use Wolfram Alpha to compute mathematical expressions or retrieve information.",
    "trigger_zapier_webhook": "Trigger a Zapier webhook to execute predefined automated workflows.",
    "send_slack_message": "Send messages to specific Slack channels to communicate with team members."
}

In [34]:
# Create embeddings for each tool description
tool_embeddings = []
tool_names = []

for tool_name, description in tool_descriptions.items():
    embedding = embeddings.embed_query(description)
    tool_embeddings.append(embedding)
    tool_names.append(tool_name)

In [35]:
# Initialize FAISS vector store
dimension = len(tool_embeddings[0])  # Assuming all embeddings have the same dimension
index = faiss.IndexFlatL2(dimension)

In [36]:
# Normalize embeddings for cosine similarity
faiss.normalize_L2(np.array(tool_embeddings).astype('float32'))

In [37]:
# Convert list to FAISS-compatible format
tool_embeddings_np = np.array(tool_embeddings).astype('float32')
index.add(tool_embeddings_np)

In [38]:
# Map index to tool functions
index_to_tool = {
    0: "query_wolfram_alpha",
    1: "trigger_zapier_webhook",
    2: "send_slack_message"
}

In [39]:
def select_tool(query: str, top_k: int = 1) -> list:
    """
    Select the most relevant tool(s) based on the user's query using vector-based retrieval.
    
    Args:
        query (str): The user's input query.
        top_k (int): Number of top tools to retrieve.
        
    Returns:
        list: List of selected tool functions.
    """
    query_embedding = np.array(embeddings.embed_query(query), dtype='float32')
    faiss.normalize_L2(query_embedding.reshape(1, -1))
    D, I = index.search(query_embedding.reshape(1, -1), top_k)
    selected_tools = [index_to_tool[idx] for idx in I[0] if idx in index_to_tool]
    return selected_tools

In [40]:
def determine_parameters(query: str, tool_name: str) -> dict:
    """
    Use the LLM to analyze the query and determine the parameters for the tool to be invoked.
    
    Args:
        query (str): The user's input query.
        tool_name (str): The selected tool name.
        
    Returns:
        dict: Parameters for the tool.
    """
    messages = [
        HumanMessage(content=f"Based on the user's query: '{query}', what parameters should be used for the tool '{tool_name}'?")
    ]
    
    # Call the LLM to extract parameters
    response = llm(messages)
    
    # Extract content from the AIMessage object
    response_content = response.content
    
    # Parse the response content as JSON
    response_data = json.loads(response_content)
    
    # Example logic to parse response from LLM
    parameters = {}
    if tool_name == "query_wolfram_alpha":
        parameters["expression"] = response_data.get('expression')  # Extract mathematical expression
    elif tool_name == "trigger_zapier_webhook":
        parameters["zap_id"] = response_data.get('zap_id', "123456")  # Default Zap ID if not provided
        parameters["payload"] = response_data.get('payload', {"data": query})
    elif tool_name == "send_slack_message":
        parameters["channel"] = response_data.get('channel', "#general")
        parameters["message"] = response_data.get('message', query)
    
    return parameters

In [41]:
# Example user query
user_query = "Solve this equation: 2x + 3 = 7"

# Select the top tool
selected_tools = select_tool(user_query, top_k=1)
tool_name = selected_tools[0] if selected_tools else None

if tool_name:
    # Use LLM to determine the parameters based on the query and the selected tool
    args = determine_parameters(user_query, tool_name)

    # Invoke the selected tool
    try:
        # Assuming each tool has an `invoke` method to execute it
        tool_result = globals()[tool_name].invoke(args)
        print(f"Tool '{tool_name}' Result: {tool_result}")
    except ValueError as e:
        print(f"Error invoking tool '{tool_name}': {e}")
else:
    print("No tool was selected.")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)